**Setup**

In [1]:
# Install optional libraries for visualization
!pip install graphviz
!apt-get install graphviz -y


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
# Imports
import matplotlib.pyplot as plt
import networkx as nx
from collections import deque
import ipywidgets as widgets
from IPython.display import display, clear_output

---
**PROJECT: Pushdown Automata Playground**
---
*Description:*

This project simulates PDAs for various problems:
   - Balanced Parentheses
   - Palindrome Detection
   - Context-Free Grammar (CFG) recognition

The project is divided among 4 team members for modular presentation.

**--------------- 1 ----------------**
 * PDA Base Class & CFG → PDA Converter
 * Description:
   - Implements the general PDA class with stack and transitions.
   - Provides a function to convert a CFG (context-free grammar) into PDA rules.
   - Contains apply_ops utility for stack operations.

In [19]:
from collections import deque

class PDA:
    def __init__(self):
        self.stack = []
        self.state = 'q0'
        self.transitions = {}
        self.accept_state = 'accept'

    def add_transition(self, state, input_char, stack_top, new_state, stack_ops):
        key = (state, input_char, stack_top)
        self.transitions.setdefault(key, []).append((new_state, stack_ops))

    def reset(self):
        self.stack = []
        self.state = 'q0'

def apply_ops(stack, ops):
    for op in ops:
        if op == 'pop' and stack:
            stack.pop()
        elif op.startswith('push'):
            stack.append(op.split(':')[1])

def cfg_to_pda(productions):
    pda = PDA()
    pda.add_transition('q0', None, None, 'q1', ['push:S'])
    for lhs, rhs in productions:
        if rhs == 'ε':
            pda.add_transition('q1', None, lhs, 'q1', ['pop'])
        else:
            ops = ['pop'] + [f'push:{c}' for c in reversed(rhs)]
            pda.add_transition('q1', None, lhs, 'q1', ops)
    for lhs, rhs in productions:
        for c in rhs:
            if c.islower():
                pda.add_transition('q1', c, c, 'q1', ['pop'])
    pda.add_transition('q1', None, None, 'accept', [])
    return pda

 **---------------- 2 ----------------**
* Balanced Parentheses PDA
* Description:
   - Implements a PDA to check for balanced brackets: (), {}, [].

In [14]:
class BalancedPDA:
    def __init__(self):
        self.stack = []

    def check(self, s):
        mapping = {')':'(', '}':'{', ']':'['}
        history = []
        for char in s:
            if char in '({[':
                self.stack.append(char)
            elif char in ')}]':
                if not self.stack or self.stack[-1] != mapping[char]:
                    history.append(self.stack[:])
                    return False, history
                self.stack.pop()
            history.append(self.stack[:])
        return len(self.stack)==0, history



In [13]:
print("=== 2: Balanced Parentheses Tests ===")
bp = BalancedPDA()
tests_bp = ["()", "{[()]}", "(]", "(()"]
for t in tests_bp:
    result, _ = bp.check(t)
    print(f"{t} -> {result}")

=== 2: Balanced Parentheses Tests ===
() -> True
{[()]} -> True
(] -> False
(() -> False


**---------------- 3 ----------------**
* Palindrome PDA
* Description:
   - Implements a PDA to check if a string is a palindrome.
   - Ignores case and non-alphanumeric characters.


In [15]:
class PalindromePDA:
    def __init__(self):
        self.stack = []

    def check(self, s):
        filtered = ''.join(c.lower() for c in s if c.isalnum())
        n = len(filtered)
        mid = n // 2
        history = []
        for i in range(mid):
            self.stack.append(filtered[i])
            history.append(self.stack[:])
        start = mid + 1 if n % 2 else mid
        for i in range(start, n):
            if not self.stack or self.stack[-1] != filtered[i]:
                history.append(self.stack[:])
                return False, history
            self.stack.pop()
            history.append(self.stack[:])
        return len(self.stack)==0, history



In [16]:
print("\n=== 3: Palindrome Tests ===")
pp = PalindromePDA()
tests_pp = ["abba", "abcba", "A man, a plan, a canal: Panama", "hello"]
for t in tests_pp:
    result, _ = pp.check(t)
    print(f"{t} -> {result}")



=== 3: Palindrome Tests ===
abba -> True
abcba -> True
A man, a plan, a canal: Panama -> True
hello -> False


**---------------- 4 ----------------**
 * CFG PDA Runner & Utilities
 * Description:
   - Implements non-deterministic PDA simulation to run CFG-based PDAs.




In [17]:


class CFGPDARunner:
    def __init__(self, pda):
        self.pda = pda

    def run(self, s):
        start_config = ('q1', tuple(['S']), 0, [])
        queue = deque([start_config])
        visited = set()
        while queue:
            state, stack, i, hist = queue.popleft()
            stack_top = stack[-1] if stack else None
            hist = hist + [(state, s[i:] if i < len(s) else "", list(stack))]
            if state == 'accept' and not stack and i == len(s):
                return True, hist
            config_id = (state, stack, i)
            if config_id in visited:
                continue
            visited.add(config_id)
            current_input = s[i] if i < len(s) else None
            for (st, sym, top), nexts in self.pda.transitions.items():
                if st == state and sym == current_input and (top == stack_top or top is None):
                    for new_state, ops in nexts:
                        new_stack = list(stack)
                        apply_ops(new_stack, ops)
                        queue.append((new_state, tuple(new_stack), i+1, hist))
            for (st, sym, top), nexts in self.pda.transitions.items():
                if st == state and sym is None and (top == stack_top or top is None):
                    for new_state, ops in nexts:
                        new_stack = list(stack)
                        apply_ops(new_stack, ops)
                        queue.append((new_state, tuple(new_stack), i, hist))
        return False, []




In [20]:
print("\n=== 1 & 4: CFG PDA Tests ===")
cfg = [("S", "aSa"), ("S", "bSb"), ("S", "ε")]
pda = cfg_to_pda(cfg)
runner = CFGPDARunner(pda)
tests_cfg = ["aa", "abba", "", "abc"]
for t in tests_cfg:
    result, _ = runner.run(t)
    print(f"{t} -> {result}")


=== 1 & 4: CFG PDA Tests ===
aa -> True
abba -> True
 -> True
abc -> False
